In [1]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from urllib.parse import quote
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
import random
import cx_Oracle
import re
from selenium.webdriver.chrome.service import Service
from openpyxl import Workbook
from selenium.common.exceptions import ElementNotInteractableException ## 추가됨
from selenium.common.exceptions import NoSuchElementException ## 추가됨
import sys ## 추가됨
import pandas as pd
import numpy as np
import os
from konlpy.tag import Okt
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
from PIL import *

In [2]:
def db_delete_det():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    print('')
    print('DB connect 성공!!')
    print('')

    cs = conn.cursor()
    
    sql = "DELETE FROM DETAIL_LINK"
    sql1 = "DELETE FROM DETAIL_KEYWORD"
    sql2 = "DELETE FROM SCRIPT_TABLE"
    
    cs.execute(sql)
    deleted_rows = cs.rowcount
    print(f"{deleted_rows} rows deleted")

    
    cs.execute(sql1)
    deleted_rows1 = cs.rowcount
    print(f"{deleted_rows1} rows deleted")
    
    cs.execute(sql2)
    deleted_rows2 = cs.rowcount
    print(f"{deleted_rows2} rows deleted")
    
    cs.close()
    conn.commit()
    conn.close()

In [3]:
def sub_select():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    # select_sub = 'SELECT * FROM SUB_KEYWORD'
    select_com = "SELECT MAIN_K || ' ' || SUB_K FROM SUB_KEYWORD"
    cs.execute(select_com)
    SUB_MAIN_C = cs.fetchall()
    
    cs.close()
    conn.close()
    sub = []
    for i in range(len(SUB_MAIN_C)):
        sub.append(SUB_MAIN_C[i][0])
    return sub

In [13]:
def df_insert_detail(ms_k = list, det_k = list):
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    print('DB 연결 성공')
    cs = conn.cursor()
    MS_K = ms_k
    DET_K = det_k
    sql = 'INSERT INTO DETAIL_KEYWORD (MAINSUB_K, DET_K) VALUES(:1, :2)'
    for i in range(len(ms_k)):
        cs.execute(sql, (MS_K[i], DET_K[i]))
        
    print('Insert 성공')
    cs.close()
    conn.commit()
    conn.close()

In [26]:
def get_detail():
    sub = select_detail()
    for i in range(len(sub)):
        keyword = sub[i][0]
        get_data_keyword(keyword)

In [10]:
def select_detail():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    select_detail = "SELECT MAINSUB_K || ' ' || DET_K FROM DETAIL_KEYWORD"
    cs.execute(select_detail)
    main_sub_detail = cs.fetchall()


    cs.close()
    conn.commit()
    conn.close()
    return main_sub_detail

In [27]:
def get_data_keyword(keyword):

    #URL 지정
    URL = f"https://www.youtube.com/results?search_query={quote(keyword)}"

    #URL 실행
    driver.get(URL)
    sleep(3)
    #필터 버튼 클릭
    driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
    sleep(1)
    
    #이번 달
    upload_date = driver.find_element(By.XPATH,'//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]')
    sleep(1)
    click_list_1 = upload_date.find_elements(By.ID,"endpoint")
    sleep(1)
    click_list_1[3].click()
    sleep(1)

    driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
    sleep(1)
    length = driver.find_element(By.XPATH,'//*[@id="collapse-content"]/ytd-search-filter-group-renderer[3]')
    sleep(1)
    click_list_2 = length.find_elements(By.ID,"endpoint")
    sleep(1)
    click_list_2[1].click()
    sleep(1)
    # scroll2()
    
    # 페이지 소스 추출
    html_source = driver.page_source
    soup_source = BeautifulSoup(html_source, 'html.parser')
    
    # 콘텐츠 모든 정보
    content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
    
    # 콘텐츠 링크만 추출
    content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))


    i = 0
    for links in content_total_link:
        link = content_total_link[i]
        detail_insert(keyword, link)
        print(keyword)
        print("링크 : ",link)
        print("-----------------------------------------------")
        i = i + 1

In [14]:
def detail_insert(keyword, link):
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()

    insert_detail = "INSERT INTO DETAIL_LINK (DET_K, LINK) VALUES (:1, :2)"
    cs.execute(insert_detail,(keyword, link))

    cs.close()
    conn.commit()
    conn.close()

In [15]:
def comment():
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    
    sql = "SELECT LINK FROM DETAIL_LINK FETCH FIRST 200 ROWS ONLY"
    sql_insert = "INSERT INTO SCRIPT_TABLE (LINK, SCRIPT) VALUES (:1,:2)"
    cs.execute(sql)
    links = cs.fetchall()
    # print(links)
    for link in links:
        URL = link[0]
            #URL 실행
        driver.get(URL)

        wb = Workbook(write_only=True)
        ws = wb.create_sheet()

        driver.implicitly_wait(5)

        time.sleep(1.5)

        driver.find_element(By.XPATH, '//*[@id="button-shape"]/button/yt-touch-feedback-shape/div').click()
        time.sleep(1)
        try:
            driver.find_element(By.XPATH, '//*[@id="items"]/ytd-menu-service-item-renderer[1]/tp-yt-paper-item').click()
            time.sleep(1)
        except NoSuchElementException:
            continue
        except ElementNotInteractableException:
            continue
        
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

         #-------------------------------------------------------------------------------------------#
        scipt_html = soup.find_all('yt-formatted-string', class_='segment-text style-scope ytd-transcript-segment-renderer')
        
        for i in range(len(scipt_html)):
            temp_comment = scipt_html[i].text
            temp_comment = clean_sent(temp_comment)
            print(temp_comment)

            cs.execute(sql_insert, (URL, temp_comment))
            if i % 50 == 0:
               conn.commit() 
        #-------------------------------------------------------------------------------------------#
    cs.close()
    conn.commit()
    conn.close()
#----------------------------------------------------------------------------------------------------------------#


In [30]:
def clean_sent(input_string):
    string_pattern = re.compile(r'[^ㄱ-힣 0-9 a-z A-Z]')
    cleansing_string = string_pattern.sub('', input_string)
    return cleansing_string

In [62]:
def get_link_list():
    sub = select_detail()
    det_k_list=[]
    for i in range(len(sub)):
        det_k_list.append(sub[i][0])
    df_l = []
    for i in det_k_list:
        sql = f"select * from detail_link where det_k like '{i}'"
        df_l.append(make_df(sql))
    link_list=[]
    for j in range(len(df_l)):
        link_list.append(df_l[j]['LINK'])
        
    return link_list

In [63]:
def make_df(sql):
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')
    cs = conn.cursor()
    sql = sql
    cs.execute(sql)
    row = cs.fetchall()
    colname = cs.description
    col = []
    for i in colname:
        col.append(i[0])
    df =  pd.DataFrame(row,columns = col)
    return df


In [64]:
def keyword_okt_noun(column):
    okt = Okt()
    arr = np.array(column)
    temp = []
    for i in range(len(arr)):
        temp.append(okt.nouns(arr[i]))
    noun = sum(temp,[])
    df = pd.DataFrame({'Noun':noun})
    noun_set = set(noun)
    noun_kind = list(noun_set)
    num=[]
    j=0
    for i in range(len(noun_kind)):
        num.append(len(df[df['Noun']==noun_kind[j]]))
        j=j+1
    #데이터프레임 생성 및 정렬
    df1 = pd.DataFrame({'Noun':noun_kind,'count':num})
    df1s = df1.sort_values('count',ascending=False)
    return df1s


In [119]:
def get_script(list):
    link_list = list
    script_df = []
    df_list=[]
    for j in range(len(link_list)):
        for i in link_list[0]:
            sql = f"select script from script_table where link like '{i}'"
            script_df.append(make_df(sql))    
        df=pd.DataFrame()
        for i in range(len(script_df)):
            df = pd.concat([df,script_df[i]])  
        df_list.append(df)
    return df_list

In [5]:
#한번만 실행하면 됨.
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient-basic-windows.x64-21.9.0.0.0dbru\instantclient_21_9")#db연결

In [31]:
if __name__ == '__main__':
    #초기 세팅
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--start-maximized')#chrome 사이즈 지정
    driver = webdriver.Chrome(chrome_options=chrome_options)
    service = Service(ChromeDriverManager().install())
    
    db_delete_det()
    
    sub_list = sub_select()
    det_list = ['증시','기준금리','부도']
    
    df_insert_detail(ms_k = sub_list, det_k = det_list)
    
    get_detail()
    
    comment()
    
    link_list = get_link_list()
    df_list = get_script(link_list)
    ncdf_list = []
    for i in range(len(df_list)):
        ncdf_list.append(keyword_okt_noun(df_list[i]['SCRIPT']))

C:\Users\effor\AppData\Local\Temp\ipykernel_7520\1409024681.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)



DB connect 성공!!

60 rows deleted
3 rows deleted
0 rows deleted
DB 연결 성공
Insert 성공
은행 위기 증시
링크 :  https://youtube.com/watch?v=wvhvqCMtW1o&pp=ygUU7J2A7ZaJIOychOq4sCDspp3si5w%3D
-----------------------------------------------
은행 위기 증시
링크 :  https://youtube.com/watch?v=WJ_j4aUgMoE&pp=ygUU7J2A7ZaJIOychOq4sCDspp3si5w%3D
-----------------------------------------------
은행 위기 증시
링크 :  https://youtube.com/watch?v=Sa66HqnIaQw&pp=ygUU7J2A7ZaJIOychOq4sCDspp3si5w%3D
-----------------------------------------------
은행 위기 증시
링크 :  https://youtube.com/watch?v=GD5LVnun4dc&pp=ygUU7J2A7ZaJIOychOq4sCDspp3si5w%3D
-----------------------------------------------
은행 위기 증시
링크 :  https://youtube.com/watch?v=ZJdMxPLYsr0&pp=ygUU7J2A7ZaJIOychOq4sCDspp3si5w%3D
-----------------------------------------------
은행 위기 증시
링크 :  https://youtube.com/watch?v=eT5DKj4PEl8&pp=ygUU7J2A7ZaJIOychOq4sCDspp3si5w%3D
-----------------------------------------------
은행 위기 증시
링크 :  https://youtube.com/watch?v=N1otdIKHBGY&pp=ygUU7J2A7ZaJI

은행 부도 부도
링크 :  https://youtube.com/watch?v=WJ_j4aUgMoE&pp=ygUU7J2A7ZaJIOu2gOuPhCDrtoDrj4Q%3D
-----------------------------------------------
은행 부도 부도
링크 :  https://youtube.com/watch?v=CYNlb0qvMMM&pp=ygUU7J2A7ZaJIOu2gOuPhCDrtoDrj4Q%3D
-----------------------------------------------
은행 부도 부도
링크 :  https://youtube.com/watch?v=2RRM9J6SUbU&pp=ygUU7J2A7ZaJIOu2gOuPhCDrtoDrj4Q%3D
-----------------------------------------------
은행 부도 부도
링크 :  https://youtube.com/watch?v=T2glvxGnRZ0&pp=ygUU7J2A7ZaJIOu2gOuPhCDrtoDrj4Q%3D
-----------------------------------------------
은행 부도 부도
링크 :  https://youtube.com/watch?v=rnA0lokiSR0&pp=ygUU7J2A7ZaJIOu2gOuPhCDrtoDrj4Q%3D
-----------------------------------------------
미국에서
버크셔 해서웨이가
연례지수총회를 했습니다 좋은 뼈
때리는 말도 있었고
경고도 있었는데 제가 정리해왔습니다
참고로 야 워렌 버핏이 올해 92세
워렌 버핏의 오른팔이라고 하는 살림과
99세입니다 99세인데 되게
정정하세요
아 제가 진짜 내년에는 꼭 가려고요
맨날 내년에는 했는데 아 진짜 코로나
이제 끝났으니까 내년에는 제가 갔다
온 경험담을 공유하는 걸 꼭 제가
말씀드리는게 저의 이제 계획입니다
근데 다녀오신 분들은 너무 부럽고
이번에도 행사를 했는데 여기 보면은
이번에 3만명 채우는 행사장은 꽉
찼습니다 그래서
참 

받으려면 이벤트에 참여해야 되거나
어떤 조건이 있는 경우가 많은데이
스위치1 앱에서는 그런 것 없이
누구나 조건 없이 미국 달러와 일본
엔화 모두 최고 수준의 환율 우대를
적용받을 수 있다고 합니다
개인적으로는 무엇보다도 따로 계좌를
만들지 않아도 된다는 점이 가장
좋았는데요 저도 기존에 쓰던 은행
계좌를 연동해서 환전해보니까 아주
편리했습니다 그래서 미국 주식용
달러뿐만 아니라
환전 및 환테크에 관심 있으신
구독자분들은 한번 이용해 보시면 좋을
것 같습니다
음악
음악
요즘 미국의 정시 그야말로
혼란 일변도죠 그죠 예 혼란
일변도입니다
저는 그 혼란 일변도를 보면서
야
시장이 진짜 뭐 이런 수가 있느냐
이런 생각이 들 정도입니다 예 여러분
이거 한번 보세요
이거를 보시면 여러분들도 놀라실
거예요 하루에
팩 웨스트 벤코프라는 미국의
지역은행입니다 주가가 또 227가
폭락했습니다
이게 뭐 좋아진다 좋아진다 이야기만
하지 이게
실제 현장에서는요 그런 일이 안
일어난다는 거죠 나쁜 일이 더
크게 벌어진다는 이야기잖아요
실제로 지표를 한번 보실래요
지표를 보시면
은행 위기 안 끝났다는 거 다시
온다는 이야기가 설득력이 있습니다
보시는 것처럼
팩 웨스트 뱅크오프의 주가 추입니다
지금
월 3월 8일만 하더라도
2668달러 가던 것이
5월 4일에는 317달러까지
떨어졌거든요 뭐 편히 주시기도 했죠
동전주가 됐죠 이때까지 떨어진 그
액수는요
8812가 폭락했습니다
그야말로 은행의 위기가
재현되는 느낌이에요
실리콘밸리 은행이 파산을 하고
퍼스트 리퍼블릭 뱅크가
파산을 하는 그런 모습을 나타냈습니다
그러다가 5월 4일에서 5월 5일
사이에 무려 817가 폭등을
했거든요
그러다가 말입니다
576달러
597달러 여기를 이제 유지를
했습니다
그러다가 오늘 다시
5월 11일 기준이죠 우리는
12일이지만 거기는 마감이
11일이니까
47달러로
편히 주식이 다시 되었습니다
주가가 완전히
완전히 무너졌다 이렇게 이야기할만큼
말입니다 많이 떨어졌어요 많이
언행주가 여기만 떨어졌을까요
그렇지 않습

드리는게 올해 제가 2월 3월 계속
나스닥은 괜찮다 나스닥은 오를거다
유튜브에도 영상 계속 남겨놨잖아
나스닥은 괜찮아 왜
얘네들은 은행이 터져도 안 터져
애들은 그냥 경기 침체 때문에 실적이
좀 안 좋겠네 조정 좀 맞겠냐고
많은게 땡이야 아이들은 근데 개박살은
얘들 개박살 나는 거지 얘들이 얘들
개박살 난다 그래가지고 지금
경기침체니 뭐니 이러면서 쭉 밀리면은
숨피는 빠져요 나스닥은 버틸 거야
이거 어떻게 보면은
괴리율이 벌어지는 거라니까 이게 괜히
벌어질 거라니까 이런 식으로 이런
부분을 좀 체크를 해 보셨으면 좋겠고
보시면 좋을 것 같습니다 이렇게 좀
다시 오기 좀 잘라 놓고요 요거는
미국 시장에서 가장 불안한 은행은
여기 터지면 됩니다
음악
네 알겠습니다
다음 질문 또 해볼게요 그 이번에
이제 그 은행 파산 이슈가 좀
있었잖아요 이거를
2008년 금융위기랑 많이들 비교를
해서 보시고 계신데 그때도 처음에
금융위기가 이제 은행이 스타트였다고
하는데 그때 금융위기하고 지금하고 또
어떻게 다르고 그때는 왜 일어났었죠
부동산 때문이라고 하더라고요
그렇죠 그 2008년 금융위기 때는
그
빅숏이라는 그 영화 혹시 보셨어요
그 사람이 그 헤지펀드 매니저죠 그
이름 갑자기 생각이 안 나는데 아무튼
그분의 생각은 뭐였냐면은 그 미국이
왜 위험하다고 느꼈을까 그 비추엇이란
영화를 보면은
미국은 사실은이 부동산에서 오는
금융이 굉장히 큽니다
한국도 마찬가지죠 한국도 뭐 아파트
대출이나 뭐 건물 대출 해 가지고
많이 레버리지를 써서 이제 투자를
하시는데
미국 같은 경우는
당연히 주택을 이제 모기지로 해서
대출을 해서
사는데이 모기지 채권을 가지고 또 그
레버리지를 쓰는 거죠이 모기지 채권을
그냥 놔두는게 아니고 이걸 가지고
미국에 있는 투자 은행들이 이걸 이제
cdo라는 이제
컬렉터럴리게이션이라고 해가지고 요걸
하나에 또 구조와 채권으로 만들어서
이걸 이제 신용 등급을 나눠 버린
거예요 그래서 이거를 이제 여러 번
레버리지를 이렇게 돌리다 보니까
기초자산은 하나인데 예를 들어

...

놨다
맨날 저런 이야기만 하고 있는 거예요
근데
채권금이 올라가는 거 어떡할 거예요
시중금리는 자금조달 비용에 따라서
나오는 거고 자금 조달 비용이
올라가게 되면 시중금리는 당연히
올라가게 되는 거죠 기준금리는 동결에
따라도 시중금리가 계속 올라갈 기미가
보이고 있다 예를 들어서 콜 금리
같은 경우가 저점 4월 17일에
34일 저점 찍고서 지금 35까지
올랐죠 여기는 계속 유지가 되고
있습니다
얘는 그대로 유지가 되고 있는데
cd 금리 어때요 지금 이미 3월
수준을 넘어섰습니다 그래서
345에서 지금 367까지 전부
다 더 올랐어요
금융채금리 지금
그래프 보시면은 저희가 저번 방송에도
이야기했고 저저번 방송에도
이야기했는데
계속 더 빨리 올라가고 있죠 그래서
지금은 거의 어마어마하게
빠르게 올라가고 있다 지금 금융체
금리는 한 달도 안 됐어요 02
4 포인트만 올라갔다
한 달 만에
6개월 같은 경우 이런 것도 보시면요
다음에
기준금리가 인상이 될 것이냐 인하가
될 것이냐 아무래도 다음에도 또
웬만하면 동결하겠죠 미국에서 6월에
금리 인상을 하지 않는 이상은
6월에 금리 인상을 하면 따라갈 수도
있습니다
그런데 그건
둘째치고
그렇다고 하더라도 다음 달에 코픽스
금리가 또 떨어질까요
다음 달에 코픽스 금리 또 떨어지면
이거는 좋다고 받아 봐야 됩니다
이거는 전구 압력이라고 밖에 생각할
수가 없어요 예금금리 올라가고
금융체금리 올라가고
cd 금리
골 금리 다 올라가는데
금리가 떨어질 리가 없습니다 이번에도
다 올라갔어요 예금금리 제외하고는 다
수치상으로 올라갔는데
단기 기준 그 다음에
신규 기준으로 코픽스금이 떨어진 거
이거 굉장히 이상한 상황이었죠 이런
상황만 제외하게 되면요 이제 시장
금리가 바닥 짚고 올라가는 상황이에요
금리 이야기 하시던 분들 금리가 더
떨어질 것 같아요
기준금리 인원 둘째치고 시중금리가
이제 더 떨어질 것 같아요
억지로 눌러서 여기까지 눌러놨는데
용수철 효과 나고 있죠 이것만
보더라도 답이 나온다
진짜
경제위기 어마어마하게 심각해서
경제성장

나왔잖아요 압구정동 신고가 하고 이제
두 분 말씀은
매물이 거의 없다 그러는데
매물보다
매매 거래가 적습니다
매물량이 많습니다 그런 팩트를 확실히
해야죠 다만 이번에 호재로 인해서
분명히 매물의 어떤 호가 상승을
올라갈 것이고 매수자들은 더
늘어나겠죠 거기서 만약에 매수자가
매물보다 높아지면
당연히 주택가격이 신고가를 계속 더
치겠죠 그런 부분이 더 중요한 것이지
토지거래
해서 그건 조직을 하건 실수요 그
부분을 우리가 봐줘야지 그래서
신고가의 개념에 대해서 우리
시청자분들이
기사에 지금 어떤 패턴들이
신고가가 있을 때는 하락거리도 있지
않습니까 그걸 같이 이제 형편성을
맞춰서 보도가 나오면 우리
시청자분들이
시장 판단에 좋으실 텐데 매수자든
매도자든
신곡하면 뜨다 보면 어떤 일이
벌었냐면
조금 어려우셔서 파셔야 되는 입장인데
가격을 고집하다 못 팔고 타임을 누릴
때가 있어요 야 실제로 이런 부분들이
우리 저 보도하시는 분들이 좀
형평성을 더 좀 투명하고 또 균형감과
있게 해주면 더 좋겠다는 말씀드리고
싶습니다
음악
박수
라스트 댄스 과연 파월은 마지막
스텝을 밟게 될까요 내일 새벽 미국
중앙은행의 기준금리 결정을 앞두고
파월의 선택에 다시 한번 이목이
집중되고 있습니다
025 포인트 또 한 번에 인상이
유력한 가운데 이번이 올해 마지막
인상 라스트 댄스가 될지 궁금합니다
미국 경제 향방 그리고 금리 전망해
보겠습니다 김두원
업라이즈 총괄 나오셨습니다네 총관님
안녕하세요
안녕하십니까
금리 결정을 추모로 비유했을 때 과연
파월이 어떤 스텝을 밟을 것인가
개인적으로 어떻게 전망하십니까
이번이 마지막 금리인사 즉 베이비스텝
025 포인트 인상할 것이다라고
보고 있습니다
025 포인트 올리면 그거는 이제
상단 그리고 기준금리 상단에 이제는
한 5까지 올라간다는 얘기잖아요
잡히지 않았으니 한 번 더 인상이
될거다라는 거는 어느 정도는 이제
이거는 시장에 반영이 된 거 같고요
조금 전에 이게 마지막
센스가 될 거 같다고 하셨는데 그럼
이제 미국 기준금이 더 이상 인상하지

글로벌 시장 연준 한국은행이
경제를 살리기 위해서 정책을
밀어붙인다라고 하면 정말 참 좋은데
그러지 못하고 있는 상황들을
여러분들하고 공부를 하고 있는 거죠
그렇기 때문에 이런 자금들이
장기적인 관점보다는 단기적인 관점에서
이어질 가능성이 크다고 하는 거고
우리 여러분들 최근에 주가주작 8개
종목 기억하시죠 왜 알았겠죠 반대
매매
지금 빚이
늘어나고 있고 이런 대출이 우리가
생활하는데 있어서 그 돈들이 쓰이고
있다면 상관이 없는데 이런 돈들이
위험자산 쪽으로 이동이 되고 있단
말이에요 만약에 미국에서 이런
부정적인 이슈가 터지고
시장이 조금이라도 빠지면 반대매매가
더 반대매매가 나타나면서 하락이
하락을 하락이 하락을 부를 수도 있는
그런 상황이 이어질 가능성이 큽니다
또 다른 기사를 보면 여러분들하고
지난번에 공부하기도 했는데
코스닥 개인 순매수
44가 b2다 이거 여러분들 정말
상당히 문제가 있는 거예요
그러니까이 부분을 여러분들하고 왜
체크를 하느냐 지금 여기에서 이제
기술력 반등이 나와서 시장이 좀
상승하면
물론 우리 구독자분들은 공부를 많이
하셨기 때문에 잘 활용할 수 있겠죠
그런데 여기서 밀리면 예전에 밀렸다가
다시 올라가던데요라는 판단보다도 이런
상황들을 기억하셨다가 주가의 흐름을
명확하게 이해를 하셔야 돼요 또 어떤
기사들이 나오냐면요
빛을 못 갚아서 한 개의 내몰린
차주들 가게 신용
증가가 되고 있는 그런 모습입니다
제가 염려하는 건 이게 끝이면 참
좋은데 어떠한 정책도
끝이라고 우리에게 얘기해 주지 않고
있다라는 거예요 그래서 여러분들하고
시장에 대한 방향이라든지
우려에 대한 공부를 계속 하고 있는데
조금이라도 긍정적인 정책이 나온다라고
하면 이게 연착륙 경착륙을 막기 위한
영창육을 위한 정책은 이미 단기성에
해당이 되는 거고 그런 중장기적인
관점에서 좋다라고 하면 여러분들하고도
좋은 공부를 할 수 있을 텐데 지금
그러지 못하고 있기 때문에 거기다가
코로나가 끝났지만 소상공인 b35조
상환한 시기가 이제 온 거죠이 부분도
터질 가능성이 크고
특

그래서 공실률이 떨어져야 되는데 지금
타이밍이 매우 안 좋게 경기가 안
좋아지면서 오히려 빅테크 회사 뿐만이
아니라 이런 그 되게 멋진 오피스를
많이 사용했었던
스타트업체들이 망하게 되면서 이게
지금 떨어질 거라고 예상을 하고
코로나 시기에 버텨왔었던이 상업용
부동산에서 이게 높게 유지가 되고
오히려 여기서 이제 실질적으로 이제
문제가 터지는 거 아니냐라는
우려가 커지게 된 거죠이 차트도
저희가 한번 예전에 보여드렸던 거
같은데 미국에서 2019년도 말과
2022년도
4분기 대비해 가지고 미국의 주요
지역인거죠 휴스턴 dolless 그
다음에 시카고 로스앤젤레스
샌프란시스코 같은 대형 이제 좀
스타트업이나 기업체들이 많이 집중되어
있는 이런 도시에 이런 데에서의
그러니까 공실률을 보게 되면은
진한 색이 지금
2019년 4분기고 그 다음 좀 옅은
색이 2022년 4분기에요 그러니까
휴스턴 같은 경우도 지금 공실률이
높아졌고 그 다음에 달라스도 높아졌고
시카고 지금 로스앤젤레스 같은 경우는
이렇게 높아졌습니다
그러니까 지금 보통 한 10
초반대였었는데 지금은
작년 말 기준으로
20 가까이 이렇게 많이 올라갔었고
샌프란시스코도 지금 어마어마하게
늘었다는 거죠 샌프란시스코에 이제
스타트업들 같은게 많이 집중되어
있었으니까
얘네들은 2019년도에 나쁘지
않았었는데
근데 지금은 거의 뭐 지금 얘네도
20 초 가까이 지금 공시를 보고
있는 거죠 이건 여러분이 다
아시겠지만
금리 때문에
스타트업들이 추가 펀딩이 안 되니까
구조조정을 하는 회사들도 있고
그렇게 하면서 좀 큰 오피스를 막
되게 멋있게 그 샌드위치도 있고 아침
점심 저녁 다 주고 뭐 이렇게 멋진
그 오피스를 사용했던 스타트업들이
추가 펀딩 안 되니까 좀 작은 대로
옮겨가거나 아니면 아예 진짜 사람들
다 잘라 가지고 정말로 그냥
뭉쳐가지고 사는
그렇게 지금 힘든 시기를 겪게 되니까
전 지역에서의 이런 구조조정이 지금
공실률이 지금 늘어나고 있는 상황인
거죠
당연히 공실률이 이렇게 늘어나다
보니까
지난주 12개월 동

거죠 그러면 후진국들 하는 그 돈들이
선진국들을 몰리는 거죠 이제 그런
현상들이 한국에는 이제 imf를 한번
겪어 봤잖아요 그런 것들 그런 것들이
이제 저개발
후진국부터
경제가 무너지기 시작하면이 세계
경제도 뭐 영향을 미칠 수도 있다 뭐
이런 시나리오가 있는 거죠 그런 걸
얘기하는 분들도 계시고
자 미국 디폴트 땐 세계 경제 위기가
온다
imf도 모건 jp모건도 경고장을
날렸다고 합니다
근데 이게 지금 미국 디폴트는 지금
워낙 많이 겪다 보니까 어떻게 보면
자산 시장에서는 이걸 크게 따지진
않고 있는 거 같긴 해요
근데 어쨌든 저거 단일 문제면은 아닐
수 있는데 미국 경제 미국의 디폴트와
은행권 위기 그리고 경제 세계적으로
빛이 늘어나고 있는 것들 그리고
인플레이션은 조금 줄었다고 하는데
이게 지금 임금 상승을 또 계속 되고
있다 보니까
언제 다시 물가상승률이 깨질지 모른다
그리고 미국은 또 금리는 인상을 했죠
즉 캐나다 같은 경우 인상을 하지는
않았지만
기대대로 떨어뜨리고 있지 않
떨어뜨리고 있지도 않습니다 그럼
결국은 그 지금 빛을 내고 있는 건
똑같다는 얘기거든요 이게 좀
줄어들어야
숨도 쉬고 할 텐데
줄어들지 않다 보니까 굉장히 힘든
거죠 미국의 가장 큰 투자 중
jp모건은
현재 전시 상황실을 가동 중이라
합니다 자 21일부터
매일 회의를 소집하고 있대요 전시처럼
너무너무 지금 중요하고 어떻게 보면
jp모건 같은 경우는
은행권 부실이 계속 이어지고 있는데
jp모건 같은 경우에는 부실 은행들을
지금 미국에
압력인지 근데 jp모건은 옛날에
2009년에
경제 위기 때 그 막 그때 위험에
빠진 은행들을
막 그때 사들였어요 배럴터는 사드렸죠
그때만 해도 뭐 미쳤다 제 피부건도
죽었다 막 했었는데
jp모건이 그때 등칠 불리면서 어떻게
보면 정말 미국 최고 전 세계 최고의
아이비 그 투자 은행이 됐잖아요 그
아마 아마 비슷한 판단했던 거 같아요
그래서 지금도 부실 은행들을 사들이고
있는데 이게 만약에 여기서 마무리가
되면은
굉장히 좋은 선택인데 뭐 최악의
신호리가

In [60]:
def get_link_list():
    sub = select_detail()
    det_k_list=[]
    for i in range(len(sub)):
        det_k_list.append(sub[i][0])
    df_l = []
    for i in det_k_list:
        sql = f"select * from detail_link where det_k like '{i}'"
        df_l.append(make_df(sql))
    link_list=[]
    for j in range(len(df_l)):
        link_list.append(df_l[j]['LINK'])
        
    return link_list

In [65]:
link_list = get_link_list()

In [111]:
len(link_list)

3

In [112]:
def get_script(list):
    link_list = list
    script_df = []
    df_list=[]
    for j in range(len(link_list)):
        for i in link_list[0]:
            sql = f"select script from script_table where link like '{i}'"
            script_df.append(make_df(sql))    
        df=pd.DataFrame()
        for i in range(len(script_df)):
            df = pd.concat([df,script_df[i]])  
        df_list.append(df)
    return df_list

In [115]:
df_list = get_script(link_list)

In [118]:
df_list[2]

,SCRIPT
0,뭔가가 약간 땀을 삐질 흘리는 이렇게
1,말하는 느낌이었는데요 버핏은 또
2,여유롭게 그렇게 말했습니다 긍정왕
3,고핏이었습니다 그래서 세상은
4,단기의 크게 초점을 맞추고 있기
...,...
492,우리나라도 마찬 가지고 그런 자정
493,작용을 통해서
494,시스템이 더 탄탄해지는 거고 저희는
495,좀 그런


In [120]:
ncdf_list = []
for i in range(len(df_list)):
    ncdf_list.append(keyword_okt_noun(df_list[i]['SCRIPT']))
print(ncdf_list)

[     Noun  count
2292    거    398
3075    것    305
1916    그    304
1250   지금    284
2707   은행    282
...   ...    ...
1825  스턴스      1
1823   답지      1
649    상정      1
1821   여론      1
1546  초대형      1

[3093 rows x 2 columns],      Noun  count
2292    거    796
3075    것    610
1916    그    608
1250   지금    568
2707   은행    564
...   ...    ...
1825  스턴스      2
1823   답지      2
649    상정      2
1821   여론      2
1546  초대형      2

[3093 rows x 2 columns],      Noun  count
2292    거   1194
3075    것    915
1916    그    912
1250   지금    852
2707   은행    846
...   ...    ...
1825  스턴스      3
1823   답지      3
649    상정      3
1821   여론      3
1546  초대형      3

[3093 rows x 2 columns]]
